In [ ]:
import torch
import numpy as np
import time
import math
import torch # Needed if the original data was from torch
import random as rn
from numpy.random import choice

#Format data

In [ ]:
file_path = 'D:/data_test/100/pdp_n20.pt'
loaded_data = None
datasets = []
processing_successful = False

try:
    print(f"Attempting to load data from '{file_path}'...")
    loaded_data = torch.load(file_path)
    print("File loaded successfully.")

    #print data structure
    print(f"\nType of loaded data: {type(loaded_data)}")
    print(f"Length of loaded data (if iterable): {len(loaded_data) if hasattr(loaded_data, '__len__') else 'Not iterable'}")
    print("First element of loaded data (if iterable):")
    if hasattr(loaded_data, '__getitem__') and len(loaded_data) > 0:
        print(loaded_data[0])
    else:
        print("Loaded data is not indexed or is empty.")

    print("\nAttempting to extract the first 5 datasets...")
    if isinstance(loaded_data, (list, tuple)):
        if len(loaded_data) >= 5:
            datasets = loaded_data[:5]
            processing_successful = True
        else:
            print(f"The loaded data contains fewer than 5 items ({len(loaded_data)} found). Assigning all available items.")
            datasets = loaded_data
            processing_successful = True
    elif isinstance(loaded_data, dict):
        print("Loaded data is a dictionary. Assuming keys might be dataset identifiers.")
        if len(loaded_data) >= 5:
             datasets = list(loaded_data.values())[:100]
             print("Successfully extracted the first 5 datasets (assuming dictionary values are datasets).")
             processing_successful = True
        else:
            print(f"The loaded dictionary contains fewer than 5 items ({len(loaded_data)} found). Assigning all available values.")
            datasets = list(loaded_data.values())
            processing_successful = True
    else:
        print("The loaded data is not a list, tuple, or dictionary. Cannot extract multiple datasets in a standard way.")
        print("Manual inspection of the loaded data structure is required.")
        processing_successful = False

except FileNotFoundError:
    print(f"\nError: The file '{file_path}' was not found.")
    processing_successful = False
except Exception as e:
    print(f"\nAn error occurred during file loading or initial extraction: {e}")
    processing_successful = False

processed_structured_datasets = []

if processing_successful and datasets:
    print("\nAttempting to process the extracted datasets...")
    for i, dataset in enumerate(datasets):
        print(f"Processing dataset {i+1}...")
        print(f"  Type of current dataset: {type(dataset)}")
        print(f"  Content of current dataset: {dataset}")


        instance_name = f"pdp_n20_test_instance_{i+1}"
        distance_matrix = np.array

In [ ]:
def process_datasets(datasets_list):
    processed_structured_datasets = []

    def calculate_distance_matrix_np(locations_coordinates_np):
        if locations_coordinates_np.size == 0:
            return np.array([])

        num_locations = locations_coordinates_np.shape[0]
        distance_matrix = np.zeros((num_locations, num_locations))

        if locations_coordinates_np.shape[1] != 2:
             print(f"Warning: Coordinates shape is unexpected: {locations_coordinates_np.shape}")
             return np.array([])

        diff = locations_coordinates_np[:, np.newaxis, :] - locations_coordinates_np[np.newaxis, :, :]
        distance_matrix = np.sqrt(np.sum(diff**2, axis=-1))

        return distance_matrix


    for i, dataset in enumerate(datasets_list):
        instance_name = dataset.get("instance_name")

        coordinates = dataset.get("coordinates")
        pickups_deliveries = dataset.get("pd_pairs")
        depot = dataset.get("depot_node_idx") # Keep depot as integer
        num_vehicles = dataset.get("num_vehicles")

        coordinates_np = np.array([])
        distance_matrix_np = np.array([])

        if coordinates is not None:
            if isinstance(coordinates, torch.Tensor):
                coordinates_np = coordinates.numpy()
            elif isinstance(coordinates, list):
                coordinates_np = np.array(coordinates)
            else:
                 print(f"Warning: 'coordinates' in dataset {i+1} is not a list or Tensor. Type: {type(coordinates)}")

            if coordinates_np.size > 0 and coordinates_np.shape[1] == 2:
                 distance_matrix_np = calculate_distance_matrix_np(coordinates_np)
            else:
                 print(f"Warning: Skipping distance matrix calculation for dataset {i+1} due to invalid coordinates or shape.")
        else:
             print(f"Warning: 'coordinates' not found in dataset {i+1}. Cannot calculate distance matrix.")

        pickups_deliveries_np = np.array([])
        if pickups_deliveries is not None:
            if isinstance(pickups_deliveries, torch.Tensor):
                pickups_deliveries_np = pickups_deliveries.numpy()
            elif isinstance(pickups_deliveries, list):
                pickups_deliveries_np = np.array(pickups_deliveries)
            else:
                print(f"Warning: 'pd_pairs' in dataset {i+1} is not a list or Tensor. Type: {type(pickups_deliveries)}")
        else:
            print(f"Warning: 'pd_pairs' not found in dataset {i+1}.")

        depot_val = depot if depot is not None else -1
        num_vehicles_val = num_vehicles if num_vehicles is not None else -1

        processed_dataset = {
            "instance_name": instance_name,
            "coordinates": coordinates_np,
            "pickups_deliveries": pickups_deliveries_np,
            "depot": depot_val,
            "num_vehicles": num_vehicles_val,
            "distance_matrix": distance_matrix_np
        }

        processed_structured_datasets.append(processed_dataset)

    return processed_structured_datasets


processed_datasets = process_datasets(datasets)

In [ ]:
processed_datasets

#Code

In [ ]:
class AntColony(object):
    def __init__(self, distances, ants, iterations, decay, q0, current_data, alpha=1, beta=1):
        self.distances = np.array(distances)
        self.pheromone = np.ones(self.distances.shape) * 0.1
        self.all_inds = range(len(distances))
        self.ants = ants
        self.iterations = iterations
        self.decay = decay
        self.q0 = q0
        self.alpha = alpha # Pheromone influence
        self.beta = beta   # Heuristic influence

        # Precompute heuristic (inverse of distance)
        dist_handled = np.copy(self.distances)
        dist_handled[dist_handled == 0] = 1e-10 # Avoid division by zero for heuristic
        self.heuristic = (1.0 / dist_handled) ** self.beta

        self.data = current_data
        self.depot = self.data["depot"]
        self.pickups_deliveries = self.data["pickups_deliveries"]
        self.precedence = {d: p for p, d in self.pickups_deliveries}
        self.required_nodes = set([self.depot])
        for p, d in self.pickups_deliveries:
            self.required_nodes.add(p)
            self.required_nodes.add(d)


    def run(self):
        global_best_path = None
        global_best_distance = np.inf
        self.global_best_path = None
        self.global_best_distance = np.inf

        stagnation_count = 0
        max_stagnation = 20

        for i in range(self.iterations):
            all_paths_data = self.allPaths()
            valid_paths_data = [(p, d) for p, d in all_paths_data if d < np.inf and self.is_valid_tour(p)]

            current_iteration_best_path = None
            current_iteration_best_distance = np.inf

            if valid_paths_data:
                current_iteration_best_path, current_iteration_best_distance = min(valid_paths_data, key=lambda x: x[1])

                if current_iteration_best_distance < self.global_best_distance:
                    self.global_best_path = current_iteration_best_path
                    self.global_best_distance = current_iteration_best_distance
                    stagnation_count = 0
                    # print(f"Iter {i+1}: New global best: {self.format_path(self.global_best_path)}, Dist: {self.global_best_distance:.2f}")
                else:
                    stagnation_count += 1
            else:
                stagnation_count += 1

            if self.global_best_path is not None:
                self.dropPheromoneGlobalBest()


            if (i + 1) % 10 == 0 :
                if self.global_best_path:
                     print(f"Iter {i+1}: Current Global Best - {self.format_path(self.global_best_path)}, Dist: {self.global_best_distance:.4f}")
                else:
                    print(f"Iter {i+1}: No valid global best path found yet.")


            if stagnation_count >= max_stagnation:
                print(f"Stopping early due to stagnation after {i+1} iterations.")
                break

        # check if current best path is valid
        if self.global_best_path and self.is_valid_tour(self.global_best_path):
            return self.global_best_path, self.global_best_distance
        else:
            # if current best path not valid, choose next best path
            all_paths_data = self.allPaths()
            valid_paths_data = [(p, d) for p, d in all_paths_data if d < np.inf and self.is_valid_tour(p)]
            if valid_paths_data:
                final_best_path, final_best_distance = min(valid_paths_data, key=lambda x: x[1])
                return final_best_path, final_best_distance
            return None, np.inf


    def dropPheromoneGlobalBest(self):
        if self.global_best_path is None or self.global_best_distance == np.inf:
            return

        pheromone_deposit = 1.0 / self.global_best_distance
        for k in range(len(self.global_best_path) - 1):
            u, v = self.global_best_path[k], self.global_best_path[k+1]
            self.pheromone[u, v] = (1 - self.decay) * self.pheromone[u,v] + self.decay * pheromone_deposit
            self.pheromone[v, u] = self.pheromone[u, v]


    def pathDistance(self, path):
        if not path or len(path) < 2:
            return np.inf

        total_distance = 0
        for i in range(len(path) - 1):
            u, v = path[i], path[i+1]
            if u < self.distances.shape[0] and v < self.distances.shape[1]:
                total_distance += self.distances[u, v]
            else:
                return np.inf # invalid path if nodes are out of bounds
        return total_distance

    def allPaths(self):
        all_paths_data = []
        for _ in range(self.ants):
            path = self.generatePath(self.depot)
            if path: # ensure path is not None
                dist = self.pathDistance(path)
                all_paths_data.append((path, dist))
            else:
                all_paths_data.append(([], np.inf))
        return all_paths_data

    def generatePath(self, start_node):
        path = [start_node]
        visited_nodes = {start_node}

        current_node = start_node
        max_path_len = len(self.all_inds) * 2

        while len(path) < max_path_len:
            # check if all tasks are done
            all_tasks_done = self.required_nodes.issubset(visited_nodes)

            next_node = self.pickMove(current_node, visited_nodes, path, all_tasks_done)

            if next_node is None: # Stuck
                return path # return incomplete path

            path.append(next_node)
            visited_nodes.add(next_node)

            prev_node = current_node
            current_node = next_node
            self.pheromone[prev_node, current_node] = (1 - 0.1) * self.pheromone[prev_node, current_node]
            self.pheromone[current_node, prev_node] = self.pheromone[prev_node, current_node] # Symmetric

            if current_node == self.depot and all_tasks_done: # returned to depot and all tasks done
                break

        # Final check if path is complete and valid
        if not (path[-1] == self.depot and self.required_nodes.issubset(set(path))):
            # Attempt to return to depot if not already there and tasks are done
            if self.required_nodes.issubset(set(path)) and path[-1] != self.depot:
                path.append(self.depot) # Force return to depot
                pass

        return path


    def pickMove(self, current_node, visited_nodes, current_path, all_tasks_done):
        # find feasible next nodes
        candidate_nodes = []
        for next_n in self.all_inds:
            if next_n == current_node:
                continue

            if all_tasks_done:
                if next_n == self.depot:
                    candidate_nodes.append(next_n)
                continue # no move if all tasks done

            if next_n == self.depot: # no visit depot if all tasks are not done
                continue

            if next_n in visited_nodes: # no re-visit visited node
                continue

            if next_n in self.precedence:
                pickup_for_next_n = self.precedence[next_n]
                if pickup_for_next_n not in visited_nodes:
                    continue

            candidate_nodes.append(next_n)

        if not candidate_nodes:
            return None # no feasible move

        # calculate probabilities for candidate nodes
        pheromone_values = np.array([self.pheromone[current_node, c_node] for c_node in candidate_nodes])
        heuristic_values = np.array([self.heuristic[current_node, c_node] for c_node in candidate_nodes])

        # combine pheromone and heuristic
        selection_values = (pheromone_values ** self.alpha) * (heuristic_values ** self.beta)
        selection_values[np.isinf(selection_values)] = 1e9 # Handle potential inf from heuristic if dist is tiny
        selection_values[np.isnan(selection_values)] = 1e-9 # Handle potential nan

        if np.sum(selection_values) == 0: # no valid move or all probabilities are zero
            if candidate_nodes:
                 return rn.choice(candidate_nodes) if candidate_nodes else None
            return None


        if rn.random() < self.q0: # exploitation: choose the best
            best_candidate_idx = np.argmax(selection_values)
            chosen_node = candidate_nodes[best_candidate_idx]
        else: # exploration: roulette wheel selection
            probabilities = selection_values / np.sum(selection_values)
            if np.any(np.isnan(probabilities)) or np.sum(probabilities) == 0: # safety check
                 return rn.choice(candidate_nodes) if candidate_nodes else None
            try:
                chosen_node = choice(candidate_nodes, 1, p=probabilities)[0]
            except ValueError:
                return rn.choice(candidate_nodes) if candidate_nodes else None


        return chosen_node


    def is_valid_tour(self, tour):
        if not tour or len(tour) < 2:
            return False
        if tour[0] != self.depot or tour[-1] != self.depot:
            return False

        visited_in_tour = set(tour)
        if not self.required_nodes.issubset(visited_in_tour):
            return False

        visited_so_far_in_path = set()
        for node in tour:
            visited_so_far_in_path.add(node)
            if node in self.precedence: # If node is a delivery
                pickup_node = self.precedence[node]
                if pickup_node not in visited_so_far_in_path:
                    # print(f"Validation Fail: Precedence. Node {node} (delivery) visited before {pickup_node} (pickup). Path: {self.format_path(tour)}")
                    return False
        return True

    def format_path(self, path):
        if not path:
            return "Empty Path"
        return " -> ".join(map(str, path))

processed_datasets = processed_datasets

total_cost_all_datasets = 0.0
total_time_all_datasets = 0.0
num_successful_runs = 0
all_run_results = []

print("Starting Ant Colony Optimization for PDVRP datasets...")

for i, dataset_params in enumerate(processed_datasets):
    print(f"\n--- Running ACS for: {dataset_params['instance_name']} ---")
    print(f"  Depot: {dataset_params['depot']}")
    print(f"  Pickups/Deliveries: {dataset_params['pickups_deliveries']}")

    # Tạo instance AntColony cho bộ dữ liệu hiện tại
    # Truyền trực tiếp `dataset_params` cho `AntColony`
    aco_solver = AntColony(
        distances=dataset_params["distance_matrix"],
        ants=40,           # Số lượng kiến
        iterations=100,    # Số vòng lặp
        decay=0.9,         # Tốc độ bay hơi pheromone toàn cục (rho) / Hệ số cập nhật cục bộ (phi)
                           # Trong ACS, decay (rho) thường dùng cho bay hơi toàn cục.
                           # Cập nhật cục bộ có thể dùng một hệ số khác (xi hoặc phi).
                           # Hiện tại, self.decay đang được dùng cho cả hai.
        q0=0.1,            # Xác suất chọn hành động tốt nhất (khai thác)
        current_data=dataset_params, # Truyền dữ liệu hiện tại vào ACO
        alpha=1.0,         # Trọng số của pheromone
        beta=1.0           # Trọng số của heuristic (khoảng cách)
    )

    start_time = time.time()
    best_path, best_distance = aco_solver.run()
    end_time = time.time()

    execution_time = end_time - start_time
    print(f"\nResults for {dataset_params['instance_name']}:")
    if best_path and best_distance < np.inf:
        # Kiểm tra lại tính hợp lệ lần cuối cùng (để chắc chắn)
        is_final_valid = aco_solver.is_valid_tour(best_path)
        final_dist_check = aco_solver.pathDistance(best_path)

        if is_final_valid and final_dist_check < np.inf:
            print(f"  Shortest Path: {aco_solver.format_path(best_path)}")
            print(f"  Total Distance: {best_distance:.4f} (Checked: {final_dist_check:.4f})")
            print(f"  Path Valid: {is_final_valid}")
            total_cost_all_datasets += best_distance
            total_time_all_datasets += execution_time
            num_successful_runs += 1
            all_run_results.append({
                "name": dataset_params['instance_name'],
                "path": best_path,
                "distance": best_distance,
                "time": execution_time,
                "status": "Success"
            })
        else:
            print("  No valid path found (or best path became invalid).")
            print(f"  Returned Path: {aco_solver.format_path(best_path)}")
            print(f"  Returned Distance: {best_distance}")
            print(f"  Validation check: {is_final_valid}, Distance check: {final_dist_check}")
            all_run_results.append({
                "name": dataset_params['instance_name'],
                "path": None,
                "distance": np.inf,
                "time": execution_time,
                "status": "Failure - Invalid path"
            })
    else:
        print("  No valid path found within the given iterations.")
        all_run_results.append({
            "name": dataset_params['instance_name'],
            "path": None,
            "distance": np.inf,
            "time": execution_time,
            "status": "Failure - No path"
        })
    print(f"  Execution time: {execution_time:.4f} seconds")

# --- Tính toán và In kết quả trung bình ---
print("\n\n--- Overall Test Set Results ---")
print(f"Total datasets processed: {len(processed_datasets)}")
print(f"Number of successful runs (valid paths found): {num_successful_runs}")

if num_successful_runs > 0:
    average_cost = total_cost_all_datasets / num_successful_runs
    average_time = total_time_all_datasets / num_successful_runs
    print(f"Average cost (distance) for successful runs: {average_cost:.4f}")
    print(f"Average time (seconds)) for successful runs: {average_time:.4f}")
else:
    print("No successful runs to calculate an average cost.")

print("\nDetailed results per instance:")
for res in all_run_results:
    if res['status'] == "Success":
        print(f"  {res['name']}: Distance = {res['distance']:.2f}, Time = {res['time']:.2f}s, Path = {aco_solver.format_path(res['path'])}")
    else:
        print(f"  {res['name']}: Status = {res['status']}, Time = {res['time']:.2f}s")

In [ ]:
class ElitistAntColony(object):
    def __init__(self, distances, data, ants, iterations, decay, elitist_weight, alpha=1, beta=1, q0=0.5, local_decay=0.1):
        self.distances = np.array(distances)
        self.num_nodes = len(distances)
        self.all_inds = range(self.num_nodes)

        # Validate data
        required_keys = ["distance_matrix", "pickups_deliveries", "depot"]
        if not all(key in data for key in required_keys):
            raise ValueError(f"Data dictionary missing required keys: {required_keys}")

        # PDVRP-specific
        self.depot = data["depot"]
        self.precedence = {d: p for p, d in data["pickups_deliveries"]}
        self.required_nodes = set([self.depot])
        for p, d in data["pickups_deliveries"]:
            self.required_nodes.add(p)
            self.required_nodes.add(d)
        self.num_required = len(self.required_nodes)

        # Initialize pheromone with greedy tour
        initial_pheromone = self._greedy_tour_pheromone()
        self.pheromone = np.ones(self.distances.shape) * initial_pheromone

        self.ants = ants
        self.iterations = iterations
        self.decay = decay
        self.elitist_weight = elitist_weight
        self.alpha = alpha
        self.beta = beta
        self.q0 = q0
        self.local_decay = local_decay

        # Precompute heuristic
        dist_handled = np.copy(self.distances)
        dist_handled[dist_handled == 0] = 1e-10
        self.heuristic = (1.0 / dist_handled) ** self.beta

        # Global best path
        self.global_best_path = None
        self.global_best_distance = np.inf

        # Logging
        self.stuck_ants = 0
        self.backtrack_events = 0
        self.iteration_stats = []

    def _greedy_tour_pheromone(self):
        """Compute initial pheromone based on a greedy PDVRP tour."""
        path = [self.depot]
        visited = {self.depot}
        required_visited = 1
        max_attempts = 10
        attempts = 0

        while required_visited < self.num_required and attempts < max_attempts:
            current = path[-1]
            feasible = []
            for next_node in range(self.num_nodes):
                if next_node == current or next_node in visited or next_node == self.depot:
                    continue
                if next_node in self.precedence and self.precedence[next_node] not in visited:
                    continue
                if self.distances[current][next_node] > 0:
                    feasible.append((next_node, self.distances[current][next_node]))

            if not feasible:
                if len(path) > 1:
                    last_node = path.pop()
                    visited.remove(last_node)
                    if last_node in self.required_nodes and last_node != self.depot:
                        required_visited -= 1
                    self.backtrack_events += 1
                    attempts += 1
                    continue
                else:
                    attempts += 1
                    break

            next_node = min(feasible, key=lambda x: x[1])[0]
            path.append(next_node)
            visited.add(next_node)
            if next_node in self.required_nodes:
                required_visited += 1
            attempts = 0

        if path[-1] != self.depot and self.distances[path[-1]][self.depot] > 0:
            path.append(self.depot)

        if self.is_valid_tour(path):
            distance = self.pathDistance(path)
            return 1.0 / distance if distance > 0 else 1.1
        return 1.1

    def run(self):
        print("Starting Elitist Ant Colony Optimization for PDVRP...")
        stagnation_count = 0
        max_stagnation = 20

        for i in range(self.iterations):
            all_paths = self.allPaths()
            valid_paths = [(p, d) for p, d in all_paths if d < np.inf]

            iteration_best_path = None
            iteration_best_distance = np.inf

            if valid_paths:
                iteration_best_path, iteration_best_distance = min(valid_paths, key=lambda x: x[1])
                if (i + 1) % 10 == 0:
                    print(f"Iteration {i+1}: Shortest Distance: {iteration_best_distance:.4f}")
                if iteration_best_distance < self.global_best_distance:
                    self.global_best_path = iteration_best_path
                    self.global_best_distance = iteration_best_distance
                    print(f"Iteration {i+1}: New Global Best Distance: {self.global_best_distance:.4f}")
                    stagnation_count = 0
                else:
                    stagnation_count += 1
            else:
                if (i + 1) % 10 == 0:
                    print(f"Iteration {i+1}: No valid path found.")
                stagnation_count += 1

            self.iteration_stats.append({
                'iteration': i + 1,
                'best_distance': iteration_best_distance if valid_paths else np.inf,
                'valid_paths': len(valid_paths),
                'stuck_ants': self.stuck_ants,
                'backtrack_events': self.backtrack_events
            })

            if stagnation_count >= max_stagnation:
                print(f"Stopping early due to stagnation after {i+1} iterations.")
                break

            self.updatePheromone(valid_paths)
            self.pheromone *= self.decay

        print("\nOptimization finished.")
        print(f"Stuck ants total: {sum(stat['stuck_ants'] for stat in self.iteration_stats)}")
        print(f"Total backtrack events: {sum(stat['backtrack_events'] for stat in self.iteration_stats)}")
        valid_path_counts = [stat['valid_paths'] for stat in self.iteration_stats]
        print(f"Average valid paths per iteration: {np.mean(valid_path_counts) if valid_path_counts else 0:.2f}")
        return self.global_best_path, self.global_best_distance

    def updatePheromone(self, valid_paths):
        """Update pheromones using elitist strategy."""
        delta_pheromone = np.zeros((self.num_nodes, self.num_nodes))

        # Global best update
        if self.global_best_path is not None and self.global_best_distance < np.inf and self.global_best_distance > 0:
            deposit = self.elitist_weight / self.global_best_distance
            for i in range(len(self.global_best_path) - 1):
                u, v = self.global_best_path[i], self.global_best_path[i + 1]
                delta_pheromone[u][v] += deposit
                delta_pheromone[v][u] = delta_pheromone[u][v]

        self.pheromone += delta_pheromone

    def pathDistance(self, path):
        if not self.is_valid_tour(path):
            return np.inf
        total_distance = 0
        try:
            for i in range(len(path) - 1):
                u, v = path[i], path[i + 1]
                dist = self.distances[u][v]
                if not np.isfinite(dist) or dist < 0:
                    return np.inf
                total_distance += dist
            return total_distance
        except:
            return np.inf

    def allPaths(self):
        all_paths = []
        self.stuck_ants = 0
        self.backtrack_events = 0
        for _ in range(self.ants):
            path = self.generatePath(self.depot)
            all_paths.append((path, self.pathDistance(path)))
        return all_paths

    def generatePath(self, start):
        path = [start]
        visited = {start}
        required_visited_count = 1
        attempts = 0
        max_attempts = 10

        while required_visited_count < self.num_required and attempts < max_attempts:
            move = self.pickMove(self.pheromone[path[-1]], self.distances[path[-1]], visited, path)
            if move is None:
                self.stuck_ants += 1
                if len(path) > 1:
                    last_node = path.pop()
                    visited.remove(last_node)
                    if last_node in self.required_nodes and last_node != self.depot:
                        required_visited_count -= 1
                    self.backtrack_events += 1
                    attempts += 1
                    continue
                else:
                    path = [start]
                    visited = {start}
                    required_visited_count = 1
                    attempts += 1
                    continue

            path.append(move)
            visited.add(move)
            if move in self.required_nodes:
                required_visited_count += 1
            attempts = 0

            # Local pheromone update
            self.pheromone[path[-2]][path[-1]] *= (1 - self.local_decay)
            self.pheromone[path[-2]][path[-1]] += self.local_decay * 0.1
            self.pheromone[path[-1]][path[-2]] = self.pheromone[path[-2]][path[-1]]

        if required_visited_count < self.num_required:
            return path

        if path[-1] != self.depot and self.distances[path[-1]][self.depot] > 0:
            path.append(self.depot)
            self.pheromone[path[-2]][self.depot] *= (1 - self.local_decay)
            self.pheromone[path[-2]][self.depot] += self.local_decay * 0.1
            self.pheromone[self.depot][path[-2]] = self.pheromone[path[-2]][self.depot]

        return path

    def pickMove(self, pheromone, dist, visited, path):
        feasible = []
        feasible_pickups = []
        current_node = path[-1]

        for i in self.all_inds:
            is_depot = (i == self.depot)
            all_required_visited = len(set(path) & self.required_nodes) == self.num_required

            if all_required_visited:
                if is_depot and current_node != self.depot and self.distances[current_node][i] > 0:
                    feasible.append(i)
                continue

            if is_depot:
                continue

            if i in visited:
                continue

            if i in self.precedence and self.precedence[i] not in path:
                continue

            is_pickup = any(p == i for p, d in data["pickups_deliveries"])
            if is_pickup:
                feasible_pickups.append(i)
            else:
                feasible.append(i)

        feasible = feasible_pickups + feasible

        if not feasible:
            return None

        feasible_indices = feasible
        pheromone_feasible = np.array([pheromone[i] for i in feasible_indices])
        heuristic_feasible = np.array([self.heuristic[path[-1], i] for i in feasible_indices])
        row_feasible = pheromone_feasible ** self.alpha * heuristic_feasible
        row_feasible[np.isnan(row_feasible)] = 0
        row_feasible[np.isinf(row_feasible)] = 0

        if row_feasible.sum() == 0:
            return rn.choice(feasible)

        if rn.random() < self.q0:
            max_feasible_idx = np.argmax(row_feasible)
            return feasible_indices[max_feasible_idx]
        else:
            probabilities_feasible = row_feasible / row_feasible.sum()
            try:
                chosen_feasible_idx = choice(range(len(feasible_indices)), 1, p=probabilities_feasible)[0]
                return feasible_indices[chosen_feasible_idx]
            except ValueError:
                return rn.choice(feasible)

    def is_valid_tour(self, tour):
        if not tour:
            return False
        if tour[0] != self.depot or tour[-1] != self.depot:
            return False
        visited_nodes_in_tour = set(tour)
        if visited_nodes_in_tour != self.required_nodes:
            return False
        visited_so_far = set()
        for i in range(len(tour)):
            current_node = tour[i]
            if current_node in self.precedence and self.precedence[current_node] not in visited_so_far:
                return False
            visited_so_far.add(current_node)
        return True

    def format_path(self, path):
        if path is None:
            return "No valid path found"
        if not path:
            return "Empty Path"
        return " -> ".join(map(str, path))


processed_datasets = processed_datasets

total_cost_all_datasets = 0.0
total_time_all_datasets = 0.0
num_successful_runs = 0
all_run_results = []

print("Starting Elitist Ant Colony Optimization for PDVRP datasets...")

for i, dataset_params in enumerate(processed_datasets):
    global data
    data = processed_datasets[i]
    print(f"\n--- Running Elitist ACO for: {dataset_params['instance_name']} ---")
    print(f"  Depot: {dataset_params['depot']}")
    print(f"  Pickups/Deliveries: {dataset_params['pickups_deliveries']}")

    aco_solver = ElitistAntColony(
        distances=dataset_params["distance_matrix"],
        data=data,
        ants=40,
        iterations=100,
        decay=0.9,
        elitist_weight=10.0,
        alpha=1.0,
        beta=1.0,
        q0=0.3,
        local_decay=0.9
    )

    start_time = time.time()
    best_path, best_distance = aco_solver.run()
    end_time = time.time()

    execution_time = end_time - start_time
    print(f"\nResults for {dataset_params['instance_name']}:")
    if best_path and best_distance < np.inf:
        is_final_valid = aco_solver.is_valid_tour(best_path)
        final_dist_check = aco_solver.pathDistance(best_path)

        if is_final_valid and final_dist_check < np.inf:
            print(f"  Shortest Path: {aco_solver.format_path(best_path)}")
            print(f"  Total Distance: {best_distance:.4f} (Checked: {final_dist_check:.4f})")
            print(f"  Path Valid: {is_final_valid}")
            total_cost_all_datasets += best_distance
            total_time_all_datasets += execution_time
            num_successful_runs += 1
            all_run_results.append({
                "name": dataset_params['instance_name'],
                "path": best_path,
                "distance": best_distance,
                "time": execution_time,
                "status": "Success"
            })
        else:
            print("  No valid path found (or best path became invalid).")
            print(f"  Returned Path: {aco_solver.format_path(best_path)}")
            print(f"  Returned Distance: {best_distance}")
            print(f"  Validation check: {is_final_valid}, Distance check: {final_dist_check}")
            all_run_results.append({
                "name": dataset_params['instance_name'],
                "path": None,
                "distance": np.inf,
                "time": execution_time,
                "status": "Failure - Invalid path"
            })
    else:
        print("  No valid path found within the given iterations.")
        all_run_results.append({
            "name": dataset_params['instance_name'],
            "path": None,
            "distance": np.inf,
            "time": execution_time,
            "status": "Failure - No path"
        })
    print(f"  Execution time: {execution_time:.4f} seconds")

print("\n\n--- Overall Test Set Results ---")
print(f"Total datasets processed: {len(processed_datasets)}")
print(f"Number of successful runs (valid paths found): {num_successful_runs}")

if num_successful_runs > 0:
    average_cost = total_cost_all_datasets / num_successful_runs
    average_time = total_time_all_datasets / num_successful_runs
    print(f"Average cost (distance) for successful runs: {average_cost:.4f}")
    print(f"Average time (seconds)) for successful runs: {average_time:.4f}")
else:
    print("No successful runs to calculate an average cost.")

print("\nDetailed results per instance:")
for res in all_run_results:
    if res['status'] == "Success":
        print(f"  {res['name']}: Distance = {res['distance']:.2f}, Time = {res['time']:.2f}s, Path = {aco_solver.format_path(res['path'])}")
    else:
        print(f"  {res['name']}: Status = {res['status']}, Time = {res['time']:.2f}s")


In [ ]:
class MMASAntColony(object):
    def __init__(self, distances, data, ants, iterations, decay, alpha=1, beta=1, q0=0.5, local_decay=0.1):
        self.distances = np.array(distances)
        self.num_nodes = len(distances)
        self.all_inds = range(self.num_nodes)

        required_keys = ["distance_matrix", "pickups_deliveries", "depot"]
        if not all(key in data for key in required_keys):
            raise ValueError(f"Data dictionary missing required keys: {required_keys}")

        self.depot = data["depot"]
        self.precedence = {d: p for p, d in data["pickups_deliveries"]}
        self.required_nodes = set([self.depot])
        for p, d in data["pickups_deliveries"]:
            self.required_nodes.add(p)
            self.required_nodes.add(d)
        self.num_required = len(self.required_nodes)

        # Initialize pheromone with greedy tour
        initial_pheromone = self._greedy_tour_pheromone()
        self.tau_max = initial_pheromone
        self.tau_min = initial_pheromone / 100
        self.pheromone = np.ones(self.distances.shape) * self.tau_max

        self.ants = ants
        self.iterations = iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.q0 = q0
        self.local_decay = local_decay

        # Precompute heuristic
        dist_handled = np.copy(self.distances)
        dist_handled[dist_handled == 0] = 1e-10
        self.heuristic = (1.0 / dist_handled) ** self.beta

        # Global best path
        self.global_best_path = None
        self.global_best_distance = np.inf

        # Logging
        self.stuck_ants = 0
        self.backtrack_events = 0
        self.iteration_stats = []

    def _greedy_tour_pheromone(self):
        path = [self.depot]
        visited = {self.depot}
        required_visited = 1
        max_attempts = 10
        attempts = 0

        while required_visited < self.num_required and attempts < max_attempts:
            current = path[-1]
            feasible = []
            for next_node in range(self.num_nodes):
                if next_node == current or next_node in visited or next_node == self.depot:
                    continue
                if next_node in self.precedence and self.precedence[next_node] not in visited:
                    continue
                if self.distances[current][next_node] > 0:
                    feasible.append((next_node, self.distances[current][next_node]))

            if not feasible:
                if len(path) > 1:
                    last_node = path.pop()
                    visited.remove(last_node)
                    if last_node in self.required_nodes and last_node != self.depot:
                        required_visited -= 1
                    self.backtrack_events += 1
                    attempts += 1
                    continue
                else:
                    attempts += 1
                    break

            next_node = min(feasible, key=lambda x: x[1])[0]
            path.append(next_node)
            visited.add(next_node)
            if next_node in self.required_nodes:
                required_visited += 1
            attempts = 0

        if path[-1] != self.depot and self.distances[path[-1]][self.depot] > 0:
            path.append(self.depot)

        if self.is_valid_tour(path):
            distance = self.pathDistance(path)
            return 1.0 / distance if distance > 0 else 1.1
        return 1.1

    def run(self):
        print("Starting Max-Min Ant System for PDVRP...")
        stagnation_count = 0
        max_stagnation = 20

        for i in range(self.iterations):
            all_paths = self.allPaths()
            valid_paths = [(p, d) for p, d in all_paths if d < np.inf]

            iteration_best_path = None
            iteration_best_distance = np.inf

            if valid_paths:
                iteration_best_path, iteration_best_distance = min(valid_paths, key=lambda x: x[1])
                if (i + 1) % 10 == 0:
                    print(f"Iteration {i+1}: Shortest Distance: {iteration_best_distance:.4f}")
                if iteration_best_distance < self.global_best_distance:
                    self.global_best_path = iteration_best_path
                    self.global_best_distance = iteration_best_distance
                    print(f"Iteration {i+1}: New Global Best Distance: {self.global_best_distance:.4f}")
                    stagnation_count = 0
                else:
                    stagnation_count += 1
            else:
                if (i + 1) % 10 == 0:
                    print(f"Iteration {i+1}: No valid path found.")
                stagnation_count += 1

            self.iteration_stats.append({
                'iteration': i + 1,
                'best_distance': iteration_best_distance if valid_paths else np.inf,
                'valid_paths': len(valid_paths),
                'stuck_ants': self.stuck_ants,
                'backtrack_events': self.backtrack_events
            })

            if stagnation_count >= max_stagnation:
                print(f"Stopping early due to stagnation after {i+1} iterations.")
                break

            self.pheromone *= self.decay
            self.updatePheromone(valid_paths)
            self.pheromone = np.clip(self.pheromone, self.tau_min, self.tau_max)

        print("\nOptimization finished.")
        print(f"Stuck ants total: {sum(stat['stuck_ants'] for stat in self.iteration_stats)}")
        print(f"Total backtrack events: {sum(stat['backtrack_events'] for stat in self.iteration_stats)}")
        valid_path_counts = [stat['valid_paths'] for stat in self.iteration_stats]
        print(f"Average valid paths per iteration: {np.mean(valid_path_counts) if valid_path_counts else 0:.2f}")
        return self.global_best_path, self.global_best_distance

    def updatePheromone(self, valid_paths):
        delta_pheromone = np.zeros((self.num_nodes, self.num_nodes))

        # Update for global best path
        if self.global_best_path is not None and self.global_best_distance < np.inf and self.global_best_distance > 0:
            deposit = 1.0 / self.global_best_distance
            for i in range(len(self.global_best_path) - 1):
                u, v = self.global_best_path[i], self.global_best_path[i + 1]
                delta_pheromone[u][v] += deposit
                delta_pheromone[v][u] = delta_pheromone[u][v]

        self.pheromone += delta_pheromone

    def pathDistance(self, path):
        if not self.is_valid_tour(path):
            return np.inf
        total_distance = 0
        try:
            for i in range(len(path) - 1):
                u, v = path[i], path[i + 1]
                dist = self.distances[u][v]
                if not np.isfinite(dist) or dist < 0:
                    return np.inf
                total_distance += dist
            return total_distance
        except:
            return np.inf

    def allPaths(self):
        all_paths = []
        self.stuck_ants = 0
        self.backtrack_events = 0
        for _ in range(self.ants):
            path = self.generatePath(self.depot)
            all_paths.append((path, self.pathDistance(path)))
        return all_paths

    def generatePath(self, start):
        path = [start]
        visited = {start}
        required_visited_count = 1
        attempts = 0
        max_attempts = 10

        while required_visited_count < self.num_required and attempts < max_attempts:
            move = self.pickMove(self.pheromone[path[-1]], self.distances[path[-1]], visited, path)
            if move is None:
                self.stuck_ants += 1
                if len(path) > 1:
                    last_node = path.pop()
                    visited.remove(last_node)
                    if last_node in self.required_nodes and last_node != self.depot:
                        required_visited_count -= 1
                    self.backtrack_events += 1
                    attempts += 1
                    continue
                else:
                    path = [start]
                    visited = {start}
                    required_visited_count = 1
                    attempts += 1
                    continue

            path.append(move)
            visited.add(move)
            if move in self.required_nodes:
                required_visited_count += 1
            attempts = 0

            # Local pheromone update
            self.pheromone[path[-2]][path[-1]] *= (1 - self.local_decay)
            self.pheromone[path[-2]][path[-1]] += self.local_decay * self.tau_min
            self.pheromone[path[-1]][path[-2]] = self.pheromone[path[-2]][path[-1]]

        if required_visited_count < self.num_required:
            return path

        if path[-1] != self.depot and self.distances[path[-1]][self.depot] > 0:
            path.append(self.depot)
            self.pheromone[path[-2]][self.depot] *= (1 - self.local_decay)
            self.pheromone[path[-2]][self.depot] += self.local_decay * self.tau_min
            self.pheromone[self.depot][path[-2]] = self.pheromone[path[-2]][self.depot]

        return path

    def pickMove(self, pheromone, dist, visited, path):
        feasible = []
        feasible_pickups = []
        current_node = path[-1]

        for i in self.all_inds:
            is_depot = (i == self.depot)
            all_required_visited = len(set(path) & self.required_nodes) == self.num_required

            if all_required_visited:
                if is_depot and current_node != self.depot and self.distances[current_node][i] > 0:
                    feasible.append(i)
                continue

            if is_depot:
                continue

            if i in visited:
                continue

            if i in self.precedence and self.precedence[i] not in path:
                continue

            is_pickup = any(p == i for p, d in data["pickups_deliveries"])
            if is_pickup:
                feasible_pickups.append(i)
            else:
                feasible.append(i)

        feasible = feasible_pickups + feasible

        if not feasible:
            return None

        feasible_indices = feasible
        pheromone_feasible = np.array([pheromone[i] for i in feasible_indices])
        heuristic_feasible = np.array([self.heuristic[path[-1], i] for i in feasible_indices])
        row_feasible = pheromone_feasible ** self.alpha * heuristic_feasible
        row_feasible[np.isnan(row_feasible)] = 0
        row_feasible[np.isinf(row_feasible)] = 0

        if row_feasible.sum() == 0:
            return rn.choice(feasible)

        if rn.random() < self.q0:
            max_feasible_idx = np.argmax(row_feasible)
            return feasible_indices[max_feasible_idx]
        else:
            probabilities_feasible = row_feasible / row_feasible.sum()
            try:
                chosen_feasible_idx = choice(range(len(feasible_indices)), 1, p=probabilities_feasible)[0]
                return feasible_indices[chosen_feasible_idx]
            except ValueError:
                return rn.choice(feasible)

    def is_valid_tour(self, tour):
        if not tour:
            return False
        if tour[0] != self.depot or tour[-1] != self.depot:
            return False
        visited_nodes_in_tour = set(tour)
        if visited_nodes_in_tour != self.required_nodes:
            return False
        visited_so_far = set()
        for i in range(len(tour)):
            current_node = tour[i]
            if current_node in self.precedence and self.precedence[current_node] not in visited_so_far:
                return False
            visited_so_far.add(current_node)
        return True

    def format_path(self, path):
        if path is None:
            return "No valid path found"
        if not path:
            return "Empty Path"
        return " -> ".join(map(str, path))

processed_datasets = processed_datasets

total_cost_all_datasets = 0.0
total_time_all_datasets = 0.0
num_successful_runs = 0
all_run_results = []

print("Starting Max-Min Ant Colony Optimization for PDVRP datasets...")

for i, dataset_params in enumerate(processed_datasets):
    data = processed_datasets[i]
    print(f"\n--- Running Max-Min ACO for: {dataset_params['instance_name']} ---")
    print(f"  Depot: {dataset_params['depot']}")
    print(f"  Pickups/Deliveries: {dataset_params['pickups_deliveries']}")

    aco_solver = MMASAntColony(
        distances=dataset_params['distance_matrix'],
        data=data,
        ants=40,
        iterations=100,
        decay=0.9,
        alpha=1.0,
        beta=1.0,
        q0=0.1,
        local_decay=0.9
    )

    start_time = time.time()
    best_path, best_distance = aco_solver.run()
    end_time = time.time()

    execution_time = end_time - start_time
    print(f"\nResults for {dataset_params['instance_name']}:")
    if best_path and best_distance < np.inf:
        is_final_valid = aco_solver.is_valid_tour(best_path)
        final_dist_check = aco_solver.pathDistance(best_path)

        if is_final_valid and final_dist_check < np.inf:
            print(f"  Shortest Path: {aco_solver.format_path(best_path)}")
            print(f"  Total Distance: {best_distance:.4f} (Checked: {final_dist_check:.4f})")
            print(f"  Path Valid: {is_final_valid}")
            total_cost_all_datasets += best_distance
            total_time_all_datasets += execution_time
            num_successful_runs += 1
            all_run_results.append({
                "name": dataset_params['instance_name'],
                "path": best_path,
                "distance": best_distance,
                "time": execution_time,
                "status": "Success"
            })
        else:
            print("  No valid path found (or best path became invalid).")
            print(f"  Returned Path: {aco_solver.format_path(best_path)}")
            print(f"  Returned Distance: {best_distance}")
            print(f"  Validation check: {is_final_valid}, Distance check: {final_dist_check}")
            all_run_results.append({
                "name": dataset_params['instance_name'],
                "path": None,
                "distance": np.inf,
                "time": execution_time,
                "status": "Failure - Invalid path"
            })
    else:
        print("  No valid path found within the given iterations.")
        all_run_results.append({
            "name": dataset_params['instance_name'],
            "path": None,
            "distance": np.inf,
            "time": execution_time,
            "status": "Failure - No path"
        })
    print(f"  Execution time: {execution_time:.4f} seconds")


print("\n\n--- Overall Test Set Results ---")
print(f"Total datasets processed: {len(processed_datasets)}")
print(f"Number of successful runs (valid paths found): {num_successful_runs}")

if num_successful_runs > 0:
    average_cost = total_cost_all_datasets / num_successful_runs
    average_time = total_time_all_datasets / num_successful_runs
    print(f"Average cost (distance) for successful runs: {average_cost:.4f}")
    print(f"Average time for successful runs: {average_time:.4f} seconds")
else:
    print("No successful runs to calculate an average cost.")

print("\nDetailed results per instance:")
for res in all_run_results:
    if res['status'] == "Success":
        print(f"  {res['name']}: Distance = {res['distance']:.2f}, Time = {res['time']:.2f}s, Path = {aco_solver.format_path(res['path'])}")
    else:
        print(f"  {res['name']}: Status = {res['status']}, Time = {res['time']:.2f}s")


In [ ]:
pip install ortools

In [ ]:
import torch
import os
import math
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import time

DISTANCE_SCALE_FACTOR = 10000
def compute_euclidean_distance_matrix_for_ortools(coordinates_tensor, scale_factor=DISTANCE_SCALE_FACTOR):
    locations = coordinates_tensor.tolist()
    num_locations = len(locations)
    distance_matrix = [[0] * num_locations for _ in range(num_locations)]
    for i in range(num_locations):
        for j in range(num_locations):
            if i == j:
                continue
            dist = math.sqrt((locations[i][0] - locations[j][0])**2 +
                             (locations[i][1] - locations[j][1])**2)
            distance_matrix[i][j] = int(round(dist * scale_factor))
    return distance_matrix


def create_data_model_from_pt_instance(pt_instance_data):
    data = {}
    coordinates = pt_instance_data['coordinates']
    data["distance_matrix"] = compute_euclidean_distance_matrix_for_ortools(coordinates, DISTANCE_SCALE_FACTOR)
    data["pickups_deliveries"] = [list(pair) for pair in pt_instance_data['pd_pairs']]
    data["num_vehicles"] = pt_instance_data.get('num_vehicles', 1)
    data["depot"] = pt_instance_data.get('depot_node_idx', 0)
    return data


def print_solution_ortools(data, manager, routing, solution, instance_name=""):
    objective_value = solution.ObjectiveValue()
    print(f"Instance: {instance_name}, Objective (Total Distance): {objective_value}")
    total_distance = 0
    used_vehicles_count = 0
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue
        used_vehicles_count +=1
        index = routing.Start(vehicle_id)
        plan_output = f"  Route for vehicle {vehicle_id}:\n    "
        route_distance = 0
        node_count_in_route = 0
        while not routing.IsEnd(index):
            node_actual_id = manager.IndexToNode(index)
            plan_output += f" {node_actual_id} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
            node_count_in_route +=1

        if node_count_in_route > 0 :
            node_actual_id = manager.IndexToNode(index)
            plan_output += f"{node_actual_id}\n"
            plan_output += f"    Distance of the route: {route_distance}m\n"
            print(plan_output)
            total_distance += route_distance
    if total_distance != objective_value and data["num_vehicles"] == 1 :
        print(f"  Warning: Sum of route distances ({total_distance}) != ObjectiveValue ({objective_value}) for {instance_name}")
    return objective_value

def main_ortools_solver(pt_instance_data, time_limit_seconds):
    data = create_data_model_from_pt_instance(pt_instance_data)
    instance_name = pt_instance_data.get('instance_name', 'UnknownInstance')

    if not data["pickups_deliveries"]:
        print(f"Warning: No pickup-delivery pairs found in instance {instance_name}.")
        return None, 0.0

    num_locations = len(data["distance_matrix"])
    if num_locations == 0:
        print(f"Error: Distance matrix is empty for instance {instance_name}.")
        return None, 0.0

    manager = pywrapcp.RoutingIndexManager(
        num_locations, data["num_vehicles"], data["depot"]
    )
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    max_unit_distance = math.sqrt(2)

    rough_max_route_distance = int(round(num_locations * max_unit_distance * DISTANCE_SCALE_FACTOR * 1.5))
    if rough_max_route_distance == 0 and num_locations > 1:
        rough_max_route_distance = 1

    routing.AddDimension(
        transit_callback_index,
        0,
        rough_max_route_distance,
        True,
        "Distance",
    )
    distance_dimension = routing.GetDimensionOrDie("Distance")

    for request in data["pickups_deliveries"]:
        pickup_node_original_id = request[0]
        delivery_node_original_id = request[1]

        if pickup_node_original_id >= num_locations or delivery_node_original_id >= num_locations \
           or pickup_node_original_id < 0 or delivery_node_original_id < 0 :
            print(f"Warning: Invalid node ID in P-D pair ({pickup_node_original_id}, {delivery_node_original_id}) for instance {instance_name}. Max node ID is {num_locations-1}. Skipping this pair.")
            continue

        pickup_index = manager.NodeToIndex(pickup_node_original_id)
        delivery_index = manager.NodeToIndex(delivery_node_original_id)

        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
        )
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index)
            <= distance_dimension.CumulVar(delivery_index)
        )

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = time_limit_seconds

    solve_start_time = time.time()
    solution = routing.SolveWithParameters(search_parameters)
    solve_duration = time.time() - solve_start_time

    if solution and solution.ObjectiveValue() >= 0:
        return solution.ObjectiveValue(), solve_duration
    else:
        return None, solve_duration


if __name__ == "__main__":
    # --- Cấu hình ---
    PT_FILE_PATH = "D:/data_test/100/pdp_n20.pt"
    OR_TOOLS_TIME_LIMIT_PER_INSTANCE = 5

    if not os.path.exists(PT_FILE_PATH):
        print(f"File dữ liệu .pt không tìm thấy: {PT_FILE_PATH}")
        exit()

    all_loaded_instances = torch.load(PT_FILE_PATH)

    if not all_loaded_instances:
        print(f"Không có instance nào trong file {PT_FILE_PATH}.")
        exit()

    instances_to_process = all_loaded_instances[:100]

    print(f"Sẽ giải {len(instances_to_process)} instance(s) từ file: {PT_FILE_PATH}")
    print(f"Giới hạn thời gian cho mỗi instance OR-Tools: {OR_TOOLS_TIME_LIMIT_PER_INSTANCE}s")
    print("-" * 40)

    all_results_ortools = []
    total_ortools_solve_time_all_instances = 0.0

    for idx, instance_data_pt in enumerate(instances_to_process):
        instance_name = instance_data_pt.get('instance_name', f'Instance_{idx}')
        print(f"Đang xử lý: {instance_name} ({idx + 1}/{len(instances_to_process)})")

        cost_or, time_or = main_ortools_solver(instance_data_pt, OR_TOOLS_TIME_LIMIT_PER_INSTANCE)

        total_ortools_solve_time_all_instances += time_or

        if cost_or is not None:
            print(f"  >> Kết quả OR-Tools: Chi phí = {cost_or/10000}, Thời gian = {time_or:.2f}s")
            all_results_ortools.append({
                'instance_name': instance_name,
                'cost': cost_or,
                'time': time_or,
                'status': 'SOLVED'
            })
        else:
            print(f"  >> OR-Tools không tìm thấy giải pháp trong thời gian giới hạn.")
            all_results_ortools.append({
                'instance_name': instance_name,
                'cost': float('inf'),
                'time': time_or,
                'status': 'TIME_LIMIT_REACHED'
            })
        print("-" * 30)

    print("\n\n--- TỔNG KẾT KẾT QUẢ OR-TOOLS ---")
    num_solved_by_ortools = sum(1 for r in all_results_ortools if r['status'] == 'SOLVED')
    num_time_limited = sum(1 for r in all_results_ortools if r['status'] == 'TIME_LIMIT_REACHED')
    print(f"Số instance giải được bởi OR-Tools: {num_solved_by_ortools} / {len(instances_to_process)}")
    print(f"Số instance đạt giới hạn thời gian: {num_time_limited} / {len(instances_to_process)}")


    if num_solved_by_ortools > 0:
        avg_cost_solved_ortools = sum(r['cost'] for r in all_results_ortools if r['status'] == 'SOLVED') / (num_solved_by_ortools * 10000)
        avg_time_solved_ortools = sum(r['time'] for r in all_results_ortools if r['status'] == 'SOLVED') / num_solved_by_ortools
        print(f"Chi phí trung bình (trên các instance giải được): {avg_cost_solved_ortools:.2f}")
        print(f"Thời gian giải trung bình (trên các instance giải được): {avg_time_solved_ortools:.2f}s")

    print(f"Tổng thời gian OR-Tools chạy cho tất cả các instance: {total_ortools_solve_time_all_instances:.2f}s")